# 估计

## 骰子问题

In [2]:
from thinkbayes import Suite
class Dice(Suite):
    def Likelihood(self, data, hypo):
        if hypo < data:
            return 0
        else:
            return 1.0 / hypo

suite = Dice([4, 6, 8, 12, 20])
# 如果转动得到6
suite.Update(6)
suite.Print()
print("==================")
# 多摇几次
for roll in [6, 8, 7, 7, 5, 4]:
    suite.Update(roll)
suite.Print()

4 0.0
6 0.392156862745
8 0.294117647059
12 0.196078431373
20 0.117647058824
4 0.0
6 0.0
8 0.943248453672
12 0.0552061280613
20 0.0015454182665


## 火车头问题
铁路上以 $1$ 到 $N$ 命名火车头。有一天你看到一个标号 $60$ 的火车头，请估计铁路上有多少火车头？

In [5]:
class Train(Suite):
    def Likelihood(self, data, hypo):
        if hypo < data:
            return 0
        else:
            return 1.0 / hypo
        
def Mean(suite):
    total = 0
    for hypo, prob in suite.Items():
        total += hypo * prob
    return total

for num in [500, 1000, 2000]:
    hypos = xrange(1, num+1)
    suite = Train(hypos)
    for data in [60, 30, 90]:
        suite.Update(data)
    # 使用后验概率的平均值来作为估计值会减少从长远来看的均方差
    print(Mean(suite))

151.849587959
164.305586423
171.338181092


在数学上，幂律表示公司的数量与公司规模成反比
$$PMF(x) \propto \left(\frac{1}{x}\right)^\alpha $$

In [12]:
from thinkbayes import Pmf
from thinkbayes import Percentile
class InvTrain(Train):
    # 构造一个服从幂律分布的先验
    def __init__(self, hypos, alpha=1.0):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, hypo**(-alpha))
            
for num in [500, 1000, 2000]:
    hypos = xrange(1, num+1)
    suite = InvTrain(hypos)
    for data in [60, 30, 90]:
        suite.Update(data)
    interval = Percentile(suite, 5), Percentile(suite, 95)
    print(Mean(suite), interval)

(130.70846986256004, (91, 235))
(133.27523137503127, (91, 242))
(133.99746308073063, (91, 243))


贝叶斯当中，有两种途径选择先验分布。
- 选择最能代表问题相关背景资料的先验概率，在这种情况下，先验被认为是“**信息**”。问题是，人们可能会使用不同的背景信息（或者进行不同的诠释），所以基于信息的先验往往显得主观。
- 另一种方法是所谓的“**无信息参考的先验**”，其目的是为了让数据说话，越没有约束越好。

有大量的数据时，先验的选择不是特别关键；如果你没有太多的参考数据，那么采用相关的背景信息就有很大区别了。